# API from skyscanner
The objective of this part is to extract data from Skyscanner about the flights from
    A. Berlin TXL to Cancun CUN
    B. Berlin TXL to Rio de Janeiro GIG 
    C. Berlin TXL to Buenos Aires EZE
For 3 different dates:
    A. 21-12-2019 - 27-12-2019
    B. 18-01-2020 - 24-01-2020
    C. 22-02-2020 - 28-02-2020

In [4]:
import pandas as pd
import requests
import json
import matplotlib
%matplotlib inline

def get_cheapest_flights(departure_date, return_date, final_destination):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/v1.0"

    payload = ('inboundDate='
               +return_date
               +'&children=0&infants=0&country=US&currency=EUR&locale=en-US&originPlace=TXL-sky&destinationPlace='
               +final_destination
               +'-sky&outboundDate='
               +departure_date+'&adults=1')
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a",
    'content-type': "application/x-www-form-urlencoded"
    }

    response = requests.request("POST", url, data=payload, headers=headers)

    key_pass = '/'.join(dict(response.headers)['Location'].split('/')[3:])

    # Now that I have the key pass for the search, I can use it for the search. This could be a function
# this part is from GET POLL SESSION RESULT

    base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/"
    url = base_url + key_pass
    querystring = {"pageIndex":"0","pageSize":"10", "sortType": "price"} # already sorting by price

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    
    iterinaries = pd.DataFrame(response.json()['Itineraries'])
    flight_agents = pd.DataFrame(response.json()['Agents'])

    legs_departure = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration', 'Stops']]
                         .rename(columns={'Id': 'OutboundLegId',
                             'Departure': 'Outbound_departure',
                             'Arrival': 'Outbound_arrival',
                             'Duration': 'Outbound_duration',
                             'Stops': 'Outbound_stops'}))

    legs_arrival = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration', 'Stops']]
                         .rename(columns={'Id': 'InboundLegId',
                             'Departure': 'Inbound_departure',
                             'Arrival': 'Inbound_arrival',
                             'Duration': 'Inbound_duration',
                             'Stops': 'Inbound_stops'}))
    df = (iterinaries
          .merge(legs_departure, on = 'OutboundLegId')
          .merge(legs_arrival, on = 'InboundLegId')
          .drop(columns=['OutboundLegId', 'InboundLegId', 'BookingDetailsLink'])
          )

    df['Agents'] = [i[0]['Agents'][0] for i in df['PricingOptions']]
    df['Price'] = [i[0]['Price'] for i in df['PricingOptions']]
    df['Link']  = [i[0]['DeeplinkUrl'] for i in df['PricingOptions']]
    df['Destination'] = [final_destination+'_'+departure_date]*len(df)

    df['Outbound_stops'] = [len(i) for i in df['Outbound_stops']]
    df['Inbound_stops'] = [len(i) for i in df['Inbound_stops']]

    flight_agents = flight_agents[['Id', 'Name']]

    df = df.rename(columns={'Agents': 'Id'})
    df = df.merge(flight_agents, on = 'Id')
    df = df.drop(columns='PricingOptions')
    return(df.sort_values('Price')[:1])


In [5]:
"""def query_search(destinations, dates, stay_length):
    df = pd.DataFrame()
    for destination in destinations:
        for date in dates:            
            search_date = (get_cheapest_flights(date
                                           , '-'
                                           .join(date
                                                 .split('-')[:-1])+'-'+str(int(date
                                                                               .split('-')[-1])+6) 
                                           , destination))
            df.append(search_date)
    return(df)

final_output = query_search(destinations1,dates1,stay_length1)"""

"def query_search(destinations, dates, stay_length):\n    df = pd.DataFrame()\n    for destination in destinations:\n        for date in dates:            \n            search_date = (get_cheapest_flights(date\n                                           , '-'\n                                           .join(date\n                                                 .split('-')[:-1])+'-'+str(int(date\n                                                                               .split('-')[-1])+6) \n                                           , destination))\n            df.append(search_date)\n    return(df)\n\nfinal_output = query_search(destinations1,dates1,stay_length1)"

In [6]:
cun_dec = get_cheapest_flights('2019-12-21', '2019-12-27', 'CUN')

In [15]:
cun_jan = get_cheapest_flights('2020-01-18', '2020-01-24', 'CUN')

In [16]:
cun_feb = get_cheapest_flights('2020-02-22', '2020-02-28', 'CUN') 

In [17]:
gig_dec = get_cheapest_flights('2019-12-21', '2019-12-27', 'GIG')   

In [18]:
gig_jan = get_cheapest_flights('2020-01-18', '2020-01-24', 'GIG')    

In [19]:
gig_feb = get_cheapest_flights('2020-02-22', '2020-02-28', 'GIG') 

In [20]:
eze_dec = get_cheapest_flights('2019-12-21', '2019-12-27', 'EZE')

In [21]:
eze_jan = get_cheapest_flights('2020-01-18', '2020-01-24', 'EZE')    

In [22]:
eze_feb = get_cheapest_flights('2020-02-22', '2020-02-28', 'EZE') 

In [7]:
final_output = (cun_dec
                .append(cun_jan)
                .append(cun_feb)
                .append(gig_dec)
                .append(gig_jan)
                .append(gig_feb)
                .append(eze_dec)
                .append(eze_jan)
                .append(eze_feb)
               )
len(final_output)

NameError: name 'cun_jan' is not defined

In [24]:
final_output.to_csv('data_thieves_skyscanner.csv')

In [26]:
pd.read_csv('data_thieves_skyscanner.csv')

,Unnamed: 0,Outbound_departure,Outbound_arrival,Outbound_duration,Outbound_stops,Inbound_departure,Inbound_arrival,Inbound_duration,Inbound_stops,Id,Price,Link,Destination,Name
0,9,2019-12-21T14:45:00,2019-12-22T10:45:00,1560,3,2019-12-27T11:45:00,2019-12-28T07:55:00,850,1,4132306,1351.54,http://partners.api.skyscanner.net/apiservices...,CUN_2019-12-21,United
1,2,2020-01-18T06:00:00,2020-01-18T20:41:00,1241,2,2020-01-24T14:47:00,2020-01-25T13:45:00,1018,2,3051889,563.45,http://partners.api.skyscanner.net/apiservices...,CUN_2020-01-18,KLM
2,1,2020-02-22T10:15:00,2020-02-22T22:45:00,1110,2,2020-02-28T16:30:00,2020-03-01T07:55:00,2005,2,4132306,750.60,http://partners.api.skyscanner.net/apiservices...,CUN_2020-02-22,United
3,0,2019-12-21T19:45:00,2019-12-22T18:40:00,1615,1,2019-12-27T20:40:00,2019-12-28T19:05:00,1105,2,4243649,1718.71,http://partners.api.skyscanner.net/apiservices...,GIG_2019-12-21,Vayama
4,4,2020-01-18T19:45:00,2020-01-19T18:40:00,1615,1,2020-01-24T20:05:00,2020-01-25T22:05:00,1320,2,3987731,765.39,http://partners.api.skyscanner.net/apiservices...,GIG_2020-01-18,Travel2Be
5,6,2020-02-22T11:45:00,2020-02-23T06:30:00,1365,2,2020-02-28T21:50:00,2020-02-29T22:50:00,1260,1,3987731,858.23,http://partners.api.skyscanner.net/apiservices...,GIG_2020-02-22,Travel2Be
6,6,2019-12-21T15:10:00,2019-12-22T09:50:00,1360,1,2019-12-27T14:20:00,2019-12-28T19:40:00,1520,1,1939318,1690.95,http://partners.api.skyscanner.net/apiservices...,EZE_2019-12-21,Air France
7,4,2020-01-18T18:55:00,2020-01-19T22:25:00,1890,2,2020-01-24T23:50:00,2020-01-26T10:05:00,1815,2,3987731,948.88,http://partners.api.skyscanner.net/apiservices...,EZE_2020-01-18,Travel2Be
8,0,2020-02-22T06:55:00,2020-02-23T09:30:00,1835,2,2020-02-28T23:50:00,2020-03-01T21:20:00,2490,2,4036115,901.35,http://partners.api.skyscanner.net/apiservices...,EZE_2020-02-22,Travelgenio
